In [20]:
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [28]:
class FocalLoss_Simple(torch.nn.Module):
    def __init__(self, gamma=2):
        super().__init__()
        self.gamma = gamma

    def forward(self, log_pred_prob_onehot, target):
        ##接在log_softmax後面，利用exp把probability 轉回來
        pred_prob_oh = torch.exp(log_pred_prob_onehot)
        ##只要抓Label的那個值
        pt = Variable(pred_prob_oh.data.gather(1, target.data.view(-1, 1)), requires_grad=True)
        ##Focal Loss加入
        modulator = (1 - pt) ** self.gamma
        ##計算Focal_loss
        mce = modulator * (-torch.log(pt))
        ##返回Batch計算值
        return mce.mean()

---

In [30]:
F.softmax(torch.tensor([1,0.7,0.8]).reshape(1,3))

/Users/chening/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[0.3907, 0.2894, 0.3199]])

In [29]:
torch.exp(F.log_softmax(torch.tensor([1,0.7,0.8]).reshape(1,3)))


/Users/chening/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[0.3907, 0.2894, 0.3199]])

---

In [50]:
input_=F.log_softmax(torch.tensor([0.4,0.1,0.8]).reshape(1,3))
target=torch.tensor([2])

/Users/chening/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [54]:
input_.double()

tensor([[-1.1733, -1.4733, -0.7733]], dtype=torch.float64)

In [48]:
Focal_loss=FocalLoss_Simple()

In [49]:
Focal_loss(input_,target)

tensor(0.2243, grad_fn=<MeanBackward0>)

---

In [156]:

import tensorflow as tf
logits = [[2,0.5,1],[0.1,1,3]]
labels = [[0.2,0.3,0.5],[0.1,0.6,0.3]]
logits_scaled = tf.nn.softmax(logits)

result1 = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=labels)
result2 = -tf.reduce_sum(labels*tf.log(logits_scaled),1)
result3 = tf.nn.softmax_cross_entropy_with_logits(logits=logits_scaled,labels=labels)

with tf.Session() as sess:
    print('直接使用API计算softmax交叉熵：')
    print(sess.run(result1))
    print('\n')
    print('利用原理计算softmax交叉熵：')
    print(sess.run(result2))
    print('\n')
    print(sess.run(labels*tf.log(logits_scaled)))
    print(sess.run(tf.nn.softmax(logits)))
    


直接使用API计算softmax交叉熵：
[1.4143689 1.6642545]


利用原理计算softmax交叉熵：
[1.4143689 1.6642545]


[[-0.09287377 -0.58931065 -0.7321844 ]
 [-0.30742547 -1.3045527  -0.05227635]]
[[0.6285317  0.14024438 0.2312239 ]
 [0.04622407 0.11369288 0.84008306]]


---

In [158]:
import tensorflow as tf

"""
Tensorflow实现何凯明的Focal Loss, 该损失函数主要用于解决分类问题中的类别不平衡
focal_loss_sigmoid: 二分类loss
focal_loss_softmax: 多分类loss
Reference Paper : Focal Loss for Dense Object Detection
"""

def focal_loss_sigmoid(labels,logits,alpha=0.25,gamma=2):
    """
    Computer focal loss for binary classification
    Args:
      labels: A int32 tensor of shape [batch_size].
      logits: A float32 tensor of shape [batch_size].
      alpha: A scalar for focal loss alpha hyper-parameter. If positive samples number
      > negtive samples number, alpha < 0.5 and vice versa.
      gamma: A scalar for focal loss gamma hyper-parameter.
    Returns:
      A tensor of the same shape as `lables`
    """
    y_pred=tf.nn.sigmoid(logits)
    labels=tf.to_float(labels)
    L=-labels*(1-alpha)*((1-y_pred)*gamma)*tf.log(y_pred)-\
      (1-labels)*alpha*(y_pred**gamma)*tf.log(1-y_pred)
    return L

def focal_loss_softmax(labels,logits,gamma=2):
    """
    Computer focal loss for multi classification
    Args:
      labels: A int32 tensor of shape [batch_size].
      logits: A float32 tensor of shape [batch_size,num_classes].
      gamma: A scalar for focal loss gamma hyper-parameter.
    Returns:
      A tensor of the same shape as `lables`
    """
    y_pred=tf.nn.softmax(logits,dim=-1) # [batch_size,num_classes]
    labels=tf.one_hot(labels,depth=y_pred.shape[1])
    L=-labels*((1-y_pred)**gamma)*tf.log(y_pred)
    L=tf.reduce_sum(L,axis=1)
    return L

if __name__ == '__main__':
    logits=tf.random_uniform(shape=[5],minval=-1,maxval=1,dtype=tf.float32)
    labels=tf.Variable([0,1,0,0,1])
    loss1=focal_loss_sigmoid(labels=labels,logits=logits)

    logits2=tf.random_uniform(shape=[5,4],minval=-1,maxval=1,dtype=tf.float32)
    labels2=tf.Variable([1,0,2,3,1])
    loss2=focal_loss_softmax(labels==labels2,logits=logits2)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print(sess.run(loss1))
        print(sess.run(loss2))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
dim is deprecated, use axis instead
[0.02738266 0.36066383 0.08493882 0.04741147 1.3124981 ]
[0.51290035 1.7430954  1.4329239  1.9272339  0.5157414 ]


In [162]:
logits2=tf.random_uniform(shape=[5,4],minval=-1,maxval=1,dtype=tf.float32)
labels2=tf.Variable([1,0,2,3,1])

In [163]:
gamma=2

In [164]:
y_pred=tf.nn.softmax(logits2,dim=-1) # [batch_size,num_classes]
labels=tf.one_hot(labels2,depth=y_pred.shape[1])
L=-labels*((1-y_pred)**gamma)*tf.log(y_pred)
L=tf.reduce_sum(L,axis=1)

In [168]:
tf.logging.set_verbosity(tf.logging.INFO)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(1-y_pred))
    print(sess.run(labels))
    print(sess.run(L))

[[0.88239956 0.86862177 0.49154782 0.7574308 ]
 [0.8032028  0.75464344 0.61760783 0.8245459 ]
 [0.7089582  0.8742371  0.90322185 0.5135828 ]
 [0.71932155 0.7609165  0.8038691  0.7158928 ]
 [0.79343116 0.8108505  0.6813462  0.71437216]]
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
[0.2274034  2.0928283  0.5307078  0.589462   0.32456163]
